In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots 
from tqdm.notebook import tqdm
import re 
pd.set_option('max_columns', None)
pd.options.display.max_colwidth = 100
from pandas.api.types import CategoricalDtype

In [ ]:
df = pd.read_csv('data.csv', index_col=0)
df.shape

First, tally the size distribution of all requested files. Then group the requests by file size and count the total number of requests and traffic for each group. By performing this analysis, we can gain insights into which types of files and how much resources are being devoted to serving these requests.

# Number of files in each size range

In [ ]:
df1 = df[['cid','bytes_returned']].groupby('cid').agg(['count','mean'])
df1.columns = df1.columns.get_level_values(1)
df1 = df1.reset_index()
df1['mean'] = df1['mean'].astype(int)
df1 = df1.rename(columns={"mean": "size"})
df1 = df1.sort_values(by=['size'])
df1.head()

In [ ]:
df1.shape

In [ ]:
df1['size'] = df1['size']/1024
df1.describe()

In [ ]:
df2 = pd.DataFrame(columns = ['size', 'count'])

def addRow(df2, l, r, name):
    df_temp = df1[(df1['size'] >= l) & (df1['size'] < r)]
    c = df_temp.count()[0]
    df2 = df2.append({'size':name, 'count':c}, ignore_index = True)
    return df2

df2 = addRow(df2, 0, 1, '<1KB')
df2 = addRow(df2, 1, 4, '1~4KB')
df2 = addRow(df2, 4, 64, '4~64KB')
df2 = addRow(df2, 64, 256, '64KB~256KB')
df2 = addRow(df2, 256, 1024, '256KB~1MB')
df2 = addRow(df2, 1024, 1024*16, '1MB~16MB')
df2 = addRow(df2, 1024*16, 10000000, '>16MB')

# df2.replace('[1,2)', '1')
# df2.replace('[10000,100000)', '[10000,+∞)')

total = df2['count'].sum()
df2['percentage'] = df2['count']/total
df2

In [ ]:
fig = px.bar(df2, x='size', y='count', text=['\n{0:1.2f}%'.format(x*100) for x in df2['percentage']])

fig.update_xaxes(title='file size')
fig.update_yaxes(title='file count')
                                                                 
fig.show()

# Number of requests and traffic in each size range

In [ ]:
df3 = df[['cid','bytes_returned']]
df3 = df3.rename(columns={"bytes_returned": "size"})
df3['size'] = df3['size']/1024 #KB
df3.head()

In [ ]:
df3['size_type'] = ''

def addSizeType(l, r, name):
    df3.loc[(df3['size'] >= l) & (df3['size'] < r), 'size_type'] = name

addSizeType(0, 1, '<1KB')
addSizeType(1, 4, '1~4KB')
addSizeType(4, 64, '4~64KB')
addSizeType(64, 256, '64KB~256KB')
addSizeType(256, 1024, '256KB~1MB')
addSizeType(1024, 1024*16, '1MB~16MB')
addSizeType(1024*16, 1024*1024*16, '>16MB')

df3.head()

In [ ]:
df4_1 = df3[['size','size_type']].groupby('size_type').agg('sum')
df4_1['size'] = df4_1['size']/pow(1024,2) # GB
df4_1 = df4_1.reset_index()

df4_2 = df3[['cid','size_type']].groupby('size_type').agg('count')
df4_2 = df4_2.reset_index()
df4_2 = df4_2.rename(columns={"cid": "count"})

df4 = df4_1.set_index('size_type').join(df4_2.set_index('size_type'))
df4 = df4.reset_index()

cat_size_order = CategoricalDtype(
    ['<1KB', '1~4KB', '4~64KB', '64KB~256KB', '256KB~1MB', '1MB~16MB', '>16MB'], 
    ordered=True
)
df4['size_type'] = df4['size_type'].astype(cat_size_order)
df4 = df4.sort_values('size_type')
df4.head()

In [ ]:
# create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])

fig.add_trace(go.Pie(
        labels=df4['size_type'],
        values=df4['size'],
        sort=False),1,2
    )
fig.add_trace(go.Pie(
        labels=df4['size_type'],
        values=df4['count'],
        sort=False),1,1
    )


# use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="Number of requests and traffic in each size range",
    # add annotations in the center of the donut pies.
    annotations=[dict(text='Request', x=0.16, y=0.5, font_size=20, showarrow=False),
                 dict(text='Traffic', x=0.82, y=0.5, font_size=20, showarrow=False)])

fig.show()